In [ ]:
from eth_utils import address
from web3 import Web3
import os
from solcx import compile_standard, install_solc
import json

In [6]:
version = '0.8.0'
install_solc(version)

Version('0.8.0')

In [7]:
smart_contract = "Tambola.sol"

with open(smart_contract, "r") as file:
    contract_file = file.read()

# Step 1: Se conneceter au réseau (Protocole RPC)
# Step 2: Ajouter son wallet
# Step 3: Déclarer l'adresse et l'ABI
# Step 4:
## Step 4.1: Get data
## Step 4.2:
### Step 4.2.1: Créer une transaction
### Step 4.2.2: Signer la transaction
### Step 4.2.3: Upload la transaction au réseau

In [8]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"Tambola.sol": {"content": contract_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version=version,  # type: ignore
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [9]:
# get bytecode
bytecode = compiled_sol["contracts"][smart_contract]["Tambola"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(compiled_sol["contracts"][smart_contract]["Tambola"]["metadata"])["output"]["abi"]

In [10]:
# set up connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
owner_address = "0xC1f9d761bc57d84c341984EcF5Aa62016081131E"
private_key = "0xb5e3c6f9a05210823672e87b921280b46a8a1fbd5e0b988297dac615adb53b10"

In [11]:
# initialize contract
Tambola = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.getTransactionCount(owner_address)

In [12]:
# set up transaction from constructor which executes when firstly
transaction = Tambola.constructor().buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": owner_address, 
        "nonce": nonce
    }
)

signed_tx = w3.eth.account.signTransaction(transaction, private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [13]:
tambola_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xf01Ed89Ae2c6B0224bA327a057799b77E946E7c3"
private_key = "0xc6c61ebea223b90af09dff583084ee5d0adc9bfd234c281b2b5c8d66fd667258"
nonce = w3.eth.getTransactionCount(payer_address)
value = 4

call_fun = tambola_sol.functions.pay().buildTransaction(
    {
        "value": int(value * 1e18),
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.signTransaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

In [14]:
w3.eth.get_balance(tx_receipt.contractAddress) / 1e18

2.0